In [1]:
from matplotlib import pyplot as plt
import pickle

sts = pickle.load(open('/home/msun415/SynTreeNet/results/viz/skeletons-valid.pkl', 'rb'))



17:30:12 rdkit INFO: Enabling RDKit 2023.09.5 jupyter extensions


In [2]:
from synnet.utils.data_utils import SkeletonSet
import random
random.seed(42)
for st in sts:
    random.shuffle(sts[st])
    sts[st] = sts[st][:10]
sk_set = SkeletonSet().load_skeletons(sts)

In [3]:
sk_set.embed_skeletons()

begin computing similarity matrix


100%|██████████| 625521/625521 [01:27<00:00, 7115.38it/s] 


begin mds


/home/msun415/miniconda3/envs/synnet/lib/python3.9/site-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


breaking at iteration 1 with stress 828759420.0285318
breaking at iteration 1 with stress 828442940.6316681
breaking at iteration 1 with stress 828414469.4267238
breaking at iteration 1 with stress 828356423.4770769


1061

In [5]:
import numpy as np 
from synnet.encoding.fingerprints import fp_4096
from synnet.utils.reconstruct_utils import fetch_oracle

mols = []
for smi in sk_set.lookup:
    for sk in sk_set.lookup[smi]:
        mols.append((sk.index, sk.tree.nodes[sk.tree_root]['smiles']))
oracle = fetch_oracle('drd2')
scores = [oracle(mol_sk[1]) for mol_sk in mols]





17:33:26 faiss.loader INFO: Loading faiss with AVX2 support.
17:33:26 faiss.loader INFO: Successfully loaded faiss with AVX2 support.


In [33]:
len(sk_set.lookup), len(mols), len(sk_set.sks)

1061

In [23]:
fp_dist = np.zeros((len(mols), len(mols)))
fps = [np.array(fp_4096(mol_sk[1])) for mol_sk in mols]
for i in range(len(mols)):
    for j in range(len(mols)):
        fp_dist[i, j] = np.abs(fps[i]-fps[j]).sum()

max_dist = sk_set.sim.max()
max_fp_dist = fp_dist.max()

dists = np.zeros((len(mols), len(mols)))
for i in range(len(mols)):
    for j in range(len(mols)):
        index1 = mols[i][0]
        index2 = mols[j][0]
        dists[i][j] = sk_set.sim[index1][index2]/max_dist + fp_dist[i][j]/max_fp_dist

In [32]:
mols[:3]

[(0,
  'Cn1c(Nc2cc([N+](=O)[O-])c(N3CCC(Cc4cccc(O)c4)CC3)cc2N)c(-c2nn[nH]n2)c(=O)n(C)c1=O'),
 (0,
  'Cn1c(Nc2cc([N+](=O)[O-])c(N3CCC(Cc4cccc(O)c4)CC3)cc2N)c(-c2nn[nH]n2)c(=O)n(C)c1=O'),
 (0,
  'Cn1c(Nc2cc([N+](=O)[O-])c(N3CCC(Cc4cccc(O)c4)CC3)cc2N)c(-c2nn[nH]n2)c(=O)n(C)c1=O')]

In [ ]:
fig = plt.Figure()
ax = fig.add_subplot(1,1,1)
ax.scatter(coords[:,0], coords[:,1], c=scores, cmap='viridis', edgecolor='k')
plt.colorbar(scatter, label='Score')